In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
# Load dataset
df = pd.read_csv('C:/Users/q/Desktop/jupyter/preprocessed_text_50000.csv', sep=';')


In [3]:
df

,processed_texts,most_popular_tag
0,компания zynga game network разработать игра f...,Интернет и СМИ
1,учёный обнаружить остров борнео десяток новый ...,Наука и техника
2,турция израиль собираться конец год подписать ...,Экономика
3,постоянный представитель россия евросоюз ес вл...,Мир
4,спикер еврокомиссия петер стано дать комментар...,Россия
...,...,...
49995,депутат государственный дума вторник октябрь п...,Россия
49996,новохопёрский район воронежский область сотруд...,Россия
49997,министр энергетика рф александр новак надеятьс...,Россия
49998,стилист назвать кудрявый причёска главный трен...,Ценности


In [4]:
# Настройка устройства (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
texts = df['processed_texts'].tolist()
labels = df['most_popular_tag'].tolist()


In [7]:
# Преобразование строковых меток в числовые
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [8]:
# Загрузка предобученного токенизатора RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

C:\Users\q\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Определение класса датасета
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Функция для вычисления метрик
def compute_metrics(preds, labels):
    preds = torch.argmax(preds, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return accuracy, f1

In [10]:
# Параметры
max_len = 512  # Используемое значение max_len
batch_size = 16  # Используемое значение batch_size
epochs = 30  # Используемое значение epochs

# Подготовка датасета и загрузчиков данных
dataset = TextDataset(texts, labels, tokenizer, max_len)

# Разделение на тренировочную и тестовую выборки
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Загрузка предобученной модели RoBERTa
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))
model = model.to(device)

# Определение оптимизатора
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\q\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
# Обучение и оценка модели
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_preds = []
    train_labels = []

    # Обучение
    for batch in tqdm(train_dataloader, desc=f'Training Epoch {epoch + 1}/{epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # # Отладочная информация
        # print(f"Input_ids size: {input_ids.size()}")
        # print(f"Attention_mask size: {attention_mask.size()}")
        # print(f"Labels size: {labels.size()}")

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(logits.detach().cpu())
        train_labels.extend(labels.detach().cpu())

    train_accuracy, train_f1 = compute_metrics(torch.stack(train_preds), torch.stack(train_labels))

    # Оценка на тестовом наборе
    model.eval()
    test_loss = 0
    test_preds = []
    test_labels = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc=f'Validation Epoch {epoch + 1}/{epochs}'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)


            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            test_loss += loss.item()
            test_preds.extend(logits.detach().cpu())
            test_labels.extend(labels.detach().cpu())

    test_accuracy, test_f1 = compute_metrics(torch.stack(test_preds), torch.stack(test_labels))

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {train_loss/len(train_dataloader):.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1: {train_f1:.4f}")
    print(f"Test Loss: {test_loss/len(test_dataloader):.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1: {test_f1:.4f}")

Validation Epoch 1/30: 100%|█████████████████████████████████████████████████████████| 625/625 [11:17<00:00,  1.08s/it]


Epoch 1/30
Train Loss: 2.6590, Train Accuracy: 0.2550, Train F1: 0.1483
Test Loss: 2.4304, Test Accuracy: 0.3458, Test F1: 0.2569


Validation Epoch 2/30: 100%|█████████████████████████████████████████████████████████| 625/625 [11:40<00:00,  1.12s/it]


Epoch 2/30
Train Loss: 2.3408, Train Accuracy: 0.3714, Train F1: 0.3027
Test Loss: 2.1616, Test Accuracy: 0.4278, Test F1: 0.3720


Validation Epoch 3/30: 100%|█████████████████████████████████████████████████████████| 625/625 [11:21<00:00,  1.09s/it]


...

Epoch 8/30
Train Loss: 1.1507, Train Accuracy: 0.6736, Train F1: 0.6569
Test Loss: 1.3971, Test Accuracy: 0.6150, Test F1: 0.5977


Validation Epoch 9/30: 100%|█████████████████████████████████████████████████████████| 625/625 [11:36<00:00,  1.11s/it]


Epoch 9/30
Train Loss: 1.0462, Train Accuracy: 0.6989, Train F1: 0.6849
Test Loss: 1.3894, Test Accuracy: 0.6131, Test F1: 0.5995


Training Epoch 10/30:   0%|                                                         | 3/2500 [00:27<6:20:39,  9.15s/it]


KeyboardInterrupt: 

In [13]:
import pickle


# Сохранение модели
model_path = 'roberta_model.pth'
torch.save(model.state_dict(), model_path)

# Сохранение токенизатора
tokenizer_path = 'roberta_tokenizer.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

# Сохранение энкодера меток
label_encoder_path = 'roberta_label_encoder.pkl'
with open(label_encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)

print("Модель, токенизатор и энкодер меток успешно сохранены.")


Модель, токенизатор и энкодер меток успешно сохранены.
